# Genie 일간 Top100 크롤링
* 전일 낮12부터 12시간동안 기준

## 라이브러리 로드

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import time, random
from datetime import datetime
import requests, os

## 필요한 파일 확인 및 설치

### 현재 경로 확인

In [2]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

In [3]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/live_genie'
os.makedirs(crawled_folder_path, exist_ok=True)

## 크롤링할 링크 생성

In [4]:
# make link
today_date = str(datetime.today().strftime("%Y%m%d"))
today_date

'20220415'

In [5]:
today_hour = str(datetime.today().strftime("%H"))
today_hour

'18'

In [6]:
link_list = []
first_link = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={today_date}&hh={today_hour}&rtm=Y&pg=1'
second_link = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd={today_date}&hh={today_hour}&rtm=Y&pg=2'
link_list.append(first_link)
link_list.append(second_link)
len(link_list)

2

## 순위 날짜

In [7]:
# 순위 날짜
today_rank_date = str(datetime.today().strftime("%Y-%m-%d"))
today_rank_date

'2022-04-15'

In [8]:
repeat_rank_date_list = []
for i in range(100):
    repeat_rank_date_list.append(today_rank_date)
len(repeat_rank_date_list)

100

## 순위

In [9]:
# rank
rank_list = []
for i in range(1, 101):
    rank_list.append(i)

## 곡 제목, 가수, 앨범

In [10]:
title_list = []
artist_list = []
album_list = []

for link in link_list:
        
    # 페이지 접속
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
    # 지니뮤직 url
    data = requests.get(link, headers=headers)
    # soup : html형식
    soup = bs(data.text, 'lxml')

    # 곡 제목
    song_info = soup.find_all('td', 'info')
    for one in song_info:
        temp = one.find('a', 'title ellipsis').text.strip()
        if '19금' in temp:
            temp = temp[3:].strip()
        title_list.append(temp)

    # 가수
    for one in song_info:
        temp = one.find('a', 'artist ellipsis').text
        artist_list.append(temp)

    # 앨범
    for one in song_info:
        temp = one.find('a', 'albumtitle ellipsis').text
        album_list.append(temp)

    time.sleep( random.uniform(0.5, 0.9) )

## df 만들기

In [11]:
# 데이터 셋으로 만들기
dict = {'날짜':repeat_rank_date_list, '순위':rank_list, '곡':title_list, '가수':artist_list, '앨범':album_list}
df = pd.DataFrame(dict)

In [12]:
df

,날짜,순위,곡,가수,앨범
0,2022-04-15,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2022-04-15,2,TOMBOY,(여자)아이들,I NEVER DIE
2,2022-04-15,3,LOVE DIVE,IVE (아이브),LOVE DIVE
3,2022-04-15,4,Feel My Rhythm,Red Velvet (레드벨벳),'The ReVe Festival 2022 - Feel My Rhythm'
4,2022-04-15,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)
...,...,...,...,...,...
95,2022-04-15,96,12 : 45 (Stripped),etham,12 : 45 (Stripped)
96,2022-04-15,97,봄 사랑 벚꽃 말고,HIGH4 & 아이유,봄 사랑 벚꽃 말고
97,2022-04-15,98,잊을 수 있을까,노을,이별을 말하는
98,2022-04-15,99,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",AKMU (악뮤),항해


## 파일 생성

In [13]:
# excel 파일로 출력
date = str(datetime.today().strftime("%Y%m%d_%H%M%S"))
file_name = 'live_genie_' + date + '.xlsx'
path = crawled_folder_path + '/'
df.to_excel(path + file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

live_genie_20220415_180602.xlsx 파일 생성 완료


In [ ]:
df.to_excel(f'\\\\192.168.100.178/d/용역/2022/2022년 음원사재기 관련 모니터링 위탁용역/음원순위_크롤링/live_genie/{file_name}', index=False, encoding='utf-8')

# 코드 마지막